# Proyecto Natural Language Processing (NLP):

In [31]:
# Librería para la declaración y uso de Data Frames:
import pandas as pd

## Paso 1 - Lectura de Datos: 

En primer lugar, es necesario **leer y guardar la información** en una variable para poder empezar a trabajar con ella.

Para ello, se ha guaradado el archivo con todos los datos en la ruta: */workspaces/NLP-clara-ab/data/raw/url_spam.csv* y se ha cargado en un Data Frame:

In [32]:
# Lectura del CSV con los datos, dada la ruta donde se guarda el archivo (se ha evitado cargar la primera columna con los índices de las filas):
df = pd.read_csv ('/workspaces/NLP-clara-ab/data/raw/url_spam.csv', sep = ",");

# Configuración de pandas para mostrar todas las columnas del DataFrame sin truncarlas al visualizarlo:
pd.set_option('display.max_columns', None);

# Se muestran las 5 primeras filas del Data Frame:
df.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


In [33]:
df['is_spam'][0]

np.True_

Una vez se ha cargado correctamente la información en el Data Frame df es interesante evaluar la **cantidad de información que se tiene**. Para ello, se recurre al atributo `shape` del Data Frame:

In [34]:
# Se utiliza el atributo shape del Data Frame para conocer cuánta información está cargada:
print(f" El archivo cargado contiene {df.shape[0]} URLs"); 

 El archivo cargado contiene 2999 URLs


Además, tal y como se ha visto en el **breve resumen** mostrado a partir de `.head()` la variable a predecir, `is_spam` tiene dos valores posibles: `True` or `False`. 

En este caso, nos interesa predecir si una URL es SPAM, es decir, **detectar el `TRUE`**. Por lo que se va a realizar un cambio de forma que, en todas las filas donde la columna `is_spam` sea `True`, se convertirán a `1` y donde haya un `False` se convertirán a `0`. 

Para no tener problemas, es importante comprobar, primero el **tipo de variable** que se encuentra en la columna `is_spam`:

In [35]:
# Se imprimen los tipos de variables de las dos columnas del DataFrame:
print(df.dtypes)

url        object
is_spam      bool
dtype: object


Ahora que se sabe que es un **boolean**, se puede realizar la conversión con la función `map()` de forma segura:

In [36]:
# Se codifica la columna objetivo:
df['is_spam'] = df['is_spam'].astype(bool).map({True: 1, False: 0})


# Se comprueba que se ha codificado correctamente: 
df.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,1
1,https://www.hvper.com/,1
2,https://briefingday.com/m/v4n3i4f3,1
3,https://briefingday.com/n/20200618/m#commentform,0
4,https://briefingday.com/fan,1


## Paso 2 - Análisis Exploratorio de Datos (Previo):

Antes de pasar el preprocesado de texto, donde se prepara al conjunto de datos para el análisis, es necesario realizar un **análisis exploratorio de los datos previo** de forma que se puedan **eliminar redundancias o datos faltantes** y no trabajar sobre ellos de forma innecesaria en el paso siguiente. 


### Paso 2.1 - Análisis de Duplicados:

In [37]:
# Se utiliza el método .duplicated() para identificar las URL repetidas dentro del DataFrame:
print(f" Hay un total de {df.duplicated().sum()} URLs duplicadas.");

 Hay un total de 630 URLs duplicadas.


Se han encontrado un total de 630 URLs duplicadas que se deben eliminar:

In [38]:
# Se eliminan los duplicados:
df_clean = df.drop_duplicates();

# Se comprueba que ya no quedan duplicados:
print(f" Hay un total de {df_clean.duplicated().sum()} URLs duplicadas.");

 Hay un total de 0 URLs duplicadas.


### Paso 2.2 - Análisis de Nulos:

In [39]:
# Se comprueba el porcentaje de filas que presentan al menos un valor nulo:
print(f" El {round(df_clean.isnull().any(axis=1).mean()*100, 2)} % de las filas presentan, al menos, un valor nulo");

 El 0.0 % de las filas presentan, al menos, un valor nulo


No tenemos ninguna fila con valores nulos.

## Paso 2 - Preprocesamiento de Texto:

Una vez ya se tiene guardado el conjunto de datos con una breve **limpieza previa**, se puede pasar a realizar un **preprocesado del texto a estudiar**, en este caso  las URLs.

El preprocesamiento de texto es un paso crucial en el análisis de información textual, dado que ayuda a **transformar la información en bruto en un formato más estructurado y útil** para modelos de Machine Learning. De esta forma, se ayuda a eliminar el *ruido* eliminando los patrones, *a priori*, irrelevantes. 

En este caso, se van a realizar xxxxxxxxxxxxxxxxxxxx pasos: 

1. Convertir todo el texto a minúscula

2. Segmentar las IRLs según sus signos de puntuación

3. Eliminar las StopWords

4. 